In [37]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import itertools
import copy
import numpy as np

In [38]:
#IMPORTANT
#ORDER OF LAYERS IN __INIT__ MUST BE SAME AS FORWARD PASS
#FOR THE RIGHT INDEX NUMBERS OF LAYERS


#TO CHECK:

#ARE RIGHT ROWS AND COLUMNS REMOVED?
#WHAT ABOUT BIASES?!


class Net(nn.Module):
    def __init__(self):
        
        #SEE NOTE ABOVE
        
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        output = F.log_softmax(x, dim=1)
        return output
    
    def add_unit(self, index_layer):
        
        #PICK LAYERS ACCORDING TO INDEX, SEE NOTE ABOVE CLASS
        
        layer_names_list = [x[0] for x in self.named_children()]
        layer_name_1 = layer_names_list[index_layer]
        layer_name_2 = layer_names_list[index_layer+1]
        
        #get current weights of layer 1
        current_weights_1 = self._modules[layer_name_1].weight.data
        current_num_nodes_1 = current_weights_1.shape[1]
        
        #get current weights of layer 2
        current_weights_2 = self._modules[layer_name_2].weight.data
        current_num_nodes_2 = current_weights_2.shape[0]
        
        #make weights for node layer 1 (add choice of init here)
        add_weights_1 = torch.zeros([1,current_num_nodes_1])
        nn.init.xavier_uniform_(add_weights_1, gain=nn.init.calculate_gain('relu'))
        
        #make new connection weights for all nodes of layer after
        add_weights_2 = torch.zeros([current_num_nodes_2,1])
        nn.init.xavier_uniform_(add_weights_2, gain=nn.init.calculate_gain('relu'))

        #merge weights
        new_weights_1 = torch.cat([current_weights_1,add_weights_1],dim=0) #add bottom row
        new_weights_2 = torch.cat([add_weights_2,current_weights_2],dim=1) #add first column

        #update layer shape - also adds bias automatically (lets do it manually?!)
                
        self._modules[layer_name_1] = nn.Linear(new_weights_1.shape[1],new_weights_1.shape[0])
        self._modules[layer_name_2] = nn.Linear(new_weights_2.shape[1],new_weights_2.shape[0])
        
        #set weight data to new values        
        self._modules[layer_name_1].weight.data = torch.tensor(new_weights_1, requires_grad=True)
        self._modules[layer_name_2].weight.data = torch.tensor(new_weights_2, requires_grad=True)     
        
    def remove_unit(self, index_layer, index_node):
        
        layer_names_list = [x[0] for x in self.named_children()]
        layer_name_1 = layer_names_list[index_layer]
        layer_name_2 = layer_names_list[index_layer+1]
        
        current_weights_1 = self._modules[layer_name_1].weight.data
        current_num_nodes_1 = current_weights_1.shape[1]
        
        current_weights_2 = self._modules[layer_name_2].weight.data
        current_num_nodes_2 = current_weights_2.shape[0]
        
        new_weights_1 = np.delete(current_weights_1,index_node,axis=0)
        new_weights_2 = np.delete(current_weights_2,index_node,axis=1)
        
        #update layer shape - also adds bias automatically (lets do it manually?!)
                
        self._modules[layer_name_1] = nn.Linear(new_weights_1.shape[1],new_weights_1.shape[0])
        self._modules[layer_name_2] = nn.Linear(new_weights_2.shape[1],new_weights_2.shape[0])
        
        #set weight data to new values        
        self._modules[layer_name_1].weight.data = torch.tensor(new_weights_1, requires_grad=True)
        self._modules[layer_name_2].weight.data = torch.tensor(new_weights_2, requires_grad=True)          
        

In [39]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    i = 0
    for batch_idx, (data, target) in enumerate(train_loader):
    #for batch_idx, (data,target) in itertools.islice(train_loader, stop=10000):
    
        #Take only 1000 samples
        i += 1
        if i >= 1000/64:
            break
            
        #model.add_unit(0) 
        model.remove_unit(0,0)
        
        print('Model sizes:')
        print(model.fc1.weight.data.shape)
        print(model.fc2.weight.data.shape)
        
            
        #normal training    
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        
        #return previously saved state, thus resetting to before training
        #model.load_state_dict(new_state_dict,strict=False)


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

class Args:
    batch_size= 64
    test_batch_size = 1000
    epochs = 2
    lr = 1
    gamma = 0.7
    no_cuda = False
    seed = 1
    log_interval = 10
    save_model = False

args=Args()

def main():
    args = Args()
    
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_dataset = datasets.MNIST('../data' , train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))    
    train_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    
    
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")
        

In [40]:
model = Net()

print('Model before adding node:')

print(model.fc1.weight.data.shape)
print(model.fc2.weight.data.shape)

model.add_unit(0)

print('Model after adding node:')

print(model.fc1.weight.data.shape)
print(model.fc2.weight.data.shape)

print('Model after removing node:')

model.remove_unit(0,0)

print(model.fc1.weight.data.shape)
print(model.fc2.weight.data.shape)


Model before adding node:
torch.Size([128, 784])
torch.Size([10, 128])


C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Model after adding node:
torch.Size([129, 784])
torch.Size([10, 129])
Model after removing node:


C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


torch.Size([128, 784])
torch.Size([10, 128])


In [41]:
main()

C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Gebruiker\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Model sizes:
torch.Size([127, 784])
torch.Size([10, 127])
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.278899
Model sizes:
torch.Size([126, 784])
torch.Size([10, 126])
Model sizes:
torch.Size([125, 784])
torch.Size([10, 125])
Model sizes:
torch.Size([124, 784])
torch.Size([10, 124])
Model sizes:
torch.Size([123, 784])
torch.Size([10, 123])
Model sizes:
torch.Size([122, 784])
torch.Size([10, 122])
Model sizes:
torch.Size([121, 784])
torch.Size([10, 121])
Model sizes:
torch.Size([120, 784])
torch.Size([10, 120])
Model sizes:
torch.Size([119, 784])
torch.Size([10, 119])
Model sizes:
torch.Size([118, 784])
torch.Size([10, 118])
Model sizes:
torch.Size([117, 784])
torch.Size([10, 117])
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.290956
Model sizes:
torch.Size([116, 784])
torch.Size([10, 116])
Model sizes:
torch.Size([115, 784])
torch.Size([10, 115])
Model sizes:
torch.Size([114, 784])
torch.Size([10, 114])
Model sizes:
torch.Size([113, 784])
torch.Size([10, 113])

Test set: Average loss: 2.3063, Accur